In [ ]:
# Độ đo Dice
# Import thư viện cần thiết
import numpy as np
from scipy.optimize import linear_sum_assignment

# Định nghĩa lại hàm load_seg_file để đọc file SEG
def load_seg_file(file_path):
    """ Đọc file SEG và trả về ma trận nhãn phân đoạn """
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Tìm vị trí dòng "data"
    data_index = None
    width = height = 0
    
    for i, line in enumerate(lines):
        if "data" in line.lower():
            data_index = i + 1  # Dữ liệu bắt đầu sau dòng "data"
            break
        elif line.startswith("width"):
            width = int(line.split()[1])
        elif line.startswith("height"):
            height = int(line.split()[1])

    if data_index is None:
        raise ValueError("⚠ Không tìm thấy phần 'data' trong file SEG!")

    # Khởi tạo ma trận ảnh với kích thước lấy từ header
    labels = np.zeros((height, width), dtype=np.int32)

    # Đọc dữ liệu phân đoạn
    for line in lines[data_index:]:
        values = line.split()
        if len(values) != 4:
            continue  # Bỏ qua dòng lỗi
        
        label, row, start_col, end_col = map(int, values)
        labels[row, start_col:end_col] = label  # Gán nhãn cho pixel

    return labels

# Định nghĩa hàm chuẩn hóa nhãn phân đoạn bằng ánh xạ Hungarian Algorithm
def match_labels(seg1, seg2):
    """
    Ánh xạ nhãn phân đoạn giữa hai ảnh sao cho cùng vùng có nhãn giống nhau.

    Tham số:
        seg1 (ndarray): Ma trận nhãn của ảnh phân đoạn 1.
        seg2 (ndarray): Ma trận nhãn của ảnh phân đoạn 2.

    Trả về:
        seg2_mapped (ndarray): Ma trận nhãn của seg2 sau khi ánh xạ cho khớp với seg1.
    """
    unique_labels1 = np.unique(seg1)
    unique_labels2 = np.unique(seg2)

    # Tạo ma trận chi phí dựa trên số lượng pixel trùng nhau giữa các nhãn
    cost_matrix = np.zeros((len(unique_labels1), len(unique_labels2)))

    for i, label1 in enumerate(unique_labels1):
        for j, label2 in enumerate(unique_labels2):
            cost_matrix[i, j] = -np.sum((seg1 == label1) & (seg2 == label2))  # Dùng giá trị âm vì cần tìm max

    # Sử dụng thuật toán Hungarian để tìm ánh xạ tối ưu
    row_ind, col_ind = linear_sum_assignment(cost_matrix)

    # Tạo ánh xạ nhãn
    label_mapping = {unique_labels2[j]: unique_labels1[i] for i, j in zip(row_ind, col_ind)}

    # Gán lại nhãn cho seg2
    seg2_mapped = np.vectorize(lambda x: label_mapping.get(x, 0))(seg2)  # Gán nhãn mới, nhãn không khớp giữ nguyên 0

    return seg2_mapped

# Định nghĩa hàm tính Dice Similarity Coefficient (DSC)
def dice_coefficient(seg1, seg2):
    """
    Tính độ đo Dice giữa hai ảnh phân đoạn.

    Tham số:
        seg1 (ndarray): Ma trận nhãn của ảnh phân đoạn 1.
        seg2 (ndarray): Ma trận nhãn của ảnh phân đoạn 2.

    Trả về:
        float: Giá trị Dice (%) từ 0 đến 100.
    """
    intersection = np.sum((seg1 == seg2) & (seg1 > 0))
    total_pixels = np.sum(seg1 > 0) + np.sum(seg2 > 0)
    
    dice_score = (2. * intersection) / total_pixels if total_pixels > 0 else 1.0
    return dice_score * 100  # Chuyển đổi thành %

# Định nghĩa đường dẫn đến hai file SEG
file_path_1 = "hinh kq/Ncut(L=D-W)_resized_image_50x50.seg"
file_path_2 = "hinh kq/Ncut_Lanczos(L=D-W)_resized_image_50x50.seg"

# Đọc dữ liệu từ hai file SEG
seg1 = load_seg_file(file_path_1)
seg2 = load_seg_file(file_path_2)

# Kiểm tra kích thước ảnh có khớp nhau không
if seg1.shape != seg2.shape:
    raise ValueError("⚠ Kích thước của hai ảnh phân đoạn không khớp!")

# Ánh xạ nhãn của seg2 để phù hợp với seg1
seg2_mapped = match_labels(seg1, seg2)

# Tính Dice Similarity Coefficient
dice_score = dice_coefficient(seg1, seg2_mapped)

# Xuất kết quả
dice_score

np.float64(95.63833174148361)